# 无须人类知识掌握围棋

[`程世东`](http://zhihu.com/people/cheng-shi-dong-47) 翻译

[`GitHub`](http://github.com/chengstone)  [`Mail`](mailto:69558140@163.com)

人工智能的一个长期目标是从“白板”开始，在挑战性领域达到超越人类的学习算法。最近，AlphaGo成为第一个在围棋中击败世界冠军的程序。AlphaGo中的树搜索使用深度神经网络评估局面和选定走子。这些神经网络是通过来自人类专家下棋的监督学习和通过从自我对弈中强化学习来训练的。在这里，我们介绍一个完全基于强化学习的算法，无须人类数据、指导或游戏规则以外的领域知识。AlphaGo成为自己的老师：训练一个神经网络，预测AlphaGo自己的走子选择。这个神经网络提高了树搜索的强度，在下一次迭代中产生更高质量的走子选择和更强的自我对弈。从“白板”开始，我们的新程序AlphaGo Zero实现了超人的表现，对之前的程序AlphaGo赢得了100-0的成绩。

在人工智能方面取得了很大的进展，使用了经过训练的监督学习系统来复制人类专家的决策。但是，专家数据往往昂贵，不可靠，或者根本无法使用。即使有可靠的数据，也可能会对以这种方式进行训练的系统的性能施加上限。相比之下，强化学习系统是根据它们自己的经验进行训练的，原则上允许它们超越人类的能力，并在缺乏人类专家的领域工作。最近，通过强化学习训练的深度神经网络，朝着这个目标快速发展。在Atari和3D虚拟环境等电脑游戏中，这些系统已经超越了人类。然而，在人类智力方面最具挑战性的领域 - 比如被广泛认为是人工智能面临的巨大挑战的围棋游戏 - 在广阔的搜索空间中需要精确和复杂的前瞻。完全通用的方法以前在这些领域没有实现人类级别的表现。

AlphaGo是第一个在围棋中实现超人表现的程序。我们称之为AlphaGo Fan的发行版在2015年10月击败了欧洲冠军樊麾。AlphaGo Fan使用了两个深度神经网络：输出走子概率的策略网络和输出局面评估的价值网络。策略网络最初是通过监督学习来训练的，以准确地预测人类专家的走子，并随后通过策略梯度强化学习进行改进。价值网络经过训练，可以预测策略网络与自己下棋的胜利者。一旦训练完成，这些网络与蒙特卡洛树搜索（MCTS）相结合，提供了一个前瞻性搜索，使用策略网络缩小高概率走子的搜索范围，并使用价值网络（与Monte-Carlo 使用快速展开策略rollout）来评估树中的局面。随后的版本，我们称之为AlphaGo Lee，采用了类似的方法（见[`方法`](#方法)），并在2016年3月击败了赢得18个国际冠军的李世石。

我们的AlphaGo Zero程序在几个重要方面与AlphaGo Fan和AlphaGo Lee 不同。首先，它是完全由自我对弈强化学习训练的，从随机下棋开始，没有任何监督或使用人类的数据。其次，它只使用黑子和白子作为输入特征。第三，它使用单一的神经网络，而不是分别使用策略和价值网络。最后，它使用更简单的树搜索，依靠这个单一的神经网络来评估局面和选择走子，而不需要执行任何rollouts（快速走子）。为了达到这些效果，我们引入了一种新的强化学习算法，在训练迭代中引入了前瞻搜索，从而实现了快速的提升和精确稳定的学习。搜索算法、训练过程和网络结构中的技术差异将在[`方法`](#方法)中描述。

## 1、AlphaGo Zero的强化学习

我们的新方法使用参数为θ的深度神经网络$f_θ$。这个神经网络把位置的原始表示形式s（棋盘状态）和它的历史走子记录作为输入，并输出走子概率和一个评估值（胜率），$(p,v)= f_θ(s)$。向量走子概率p表示选择每个走子的概率，$p_a  = Pr(a|s)$。评估值v是标量，估计当前玩家从局面s获胜的概率。这个神经网络将策略网络和价值网络的任务结合到一个体系结构中。神经网络由许多残差块卷积层组成，包含批归一化和整流非线性（见[`方法`](#方法)）。

AlphaGo Zero中的神经网络是通过一种新的强化学习算法从自我对弈中训练出来的。在每个局面s，由神经网络$f_θ$的预测结果作为参考执行MCTS搜索。MCTS搜索输出每个可能走子的概率π。MCTS搜索输出的概率通常比神经网络$f_θ(s)$输出的原始概率p选择更强的走子；因此，MCTS可被视为一个强有力的策略提升器。搜索 - 使用基于MCTS提升的策略来选择每个走子，然后使用获胜者z作为价值- 可以被视为一个强大的策略评估器。强化学习算法的主要思想是在策略迭代过程中重复使用这些搜索：不断更新神经网络的参数以使得网络输出的走子概率和胜率$(p,v)= f_θ (s)$，更接近地匹配MCTS提升的走子概率和自对弈胜者(π, z)；这些新的参数被用在下一次自我对弈中，使得MCTS搜索更加强大。图1显示了自我对弈训练过程。

![图1](assets\a1.png")
图1：AlphaGo Zero中的自我对弈强化学习。a 程序自我对弈状态$s_1$, ..., $s_T$。在每个局面$s_t$，使用最新的神经网络$f_θ$执行蒙特卡洛树搜索（MCTS）$α_θ$（见图2）。根据由MCTS计算的概率选择走子，$a_t$∼ $π_t$。按照游戏规则对棋局结束状态$s_T$进行评分得到游戏获胜者z。b 训练AlphaGo Zero神经网络。神经网络将棋局原始走子状态$s_t$作为其输入，将其输入给具有参数θ的多个卷积层，并且输出表示走子概率分布的向量$p_t$和表示当前玩家在局面$s_t$的获胜概率标量值$v_t$。更新神经网络的参数θ以使策略向量$p_t$与搜索概率$π_t$的相似度最大化，并使得预测的获胜者$v_t$和游戏胜者z之间的误差最小化（参见等式1）。新参数用于下一次迭代的自我对弈a中。

蒙特卡洛树搜索使用神经网络$f_θ$来指导其搜索（见图2）。搜索树中的每个边(s, a)存储先验概率P(s, a)，访问计数N(s, a)和行动价值Q(s, a)。每次搜索从根状态开始并且迭代地选择使置信上限区间$Q(s, a) +U(s, a)$最大化的走子，其中$U(s, a) ∝ P(s, a)/(1 + N(s, a))$，直到遇到叶节点$s'$。叶子节点只会被神经网络扩展和评估一次，以产生先验概率和胜率评估值，$(P(s', ·), V (s')) = f_θ(s')$。更新搜索中遍历的每个边(s, a)，增加其访问次数N(s, a)，将它的行动价值更新为在这些搜索上的平均估计值，即 $Q(s,a) = 1 / N (s,a)∑_{s’ |s,a→s’} V(s’)$，其中$s,a →s'$表示从局面s选择走子a后搜索最终达到$s'$。

![a2](assets\a2.png")
图2：AlphaGo Zero中的蒙特卡洛树搜索。a.每次模拟通过选择具有最大行动价值Q的边加上取决于所存储的先验概率P和该边的访问计数N（每次访问都被增加一次）的上限置信区间U来遍历树。b.展开叶子节点，通过神经网络$(P(s, ·), V (s)) = f_θ(s)$来评估局面s；向量P的值存储在叶子结点扩展的边上。c.更新行动价值Q等于在该行动下的子树中的所有评估值V的均值。d.一旦MCTS搜索完成，返回局面s下的落子概率π，与$N^{1 /τ}$成正比，其中N是从根状态每次移动的访问计数， τ是控制温度的参数。

MCTS可以被看作一种自我对弈算法，在给定神经网络参数θ和根状态s的情况下，计算推荐走子的先验概率向量$π = α_θ(s)$，与每次走子的指数访问计数成正比 ，$π_a  ∝ N(s,a)^{1 /τ}$，其中τ是温度参数。

神经网络通过自我对弈强化学习算法进行训练，该算法使用MCTS下每一步棋。首先，神经网络随机初始化权重$θ_0$。在随后的每次迭代i≥1时，生成自我对弈棋局（图1a）。在每个时间步t，使用前一次迭代的神经网络$f_{θ_{i-1} }$执行MCTS搜索$π_t=α_{θ_{i-1} }(s_t)$，并且通过对搜索概率$π_t$进行采样来执行走子。双方都选择跳过；当搜索值低于认输阈值时，或当游戏超过最大长度（棋盘无处走子）时，游戏在步骤T终止；然后对游戏进行评分给出最终奖励$r_T ∈ ${−1, +1}（详见[`方法`](#方法)）。MCTS搜索过程中每个时间步t的数据存储为($s_t$ , $π_t$ , $z_t$)，其中$z_t=± r_T$是当前玩家在步骤t的角度的游戏获胜者。同时（图1b），新的网络参数$θ_i$从上一次自我对弈的所有时间步产生的数据(s, π, z)中采样进行训练。调整神经网络$(p,v)= f_{θ_i } (s)$以最小化预测胜率v和自我对弈的胜者z之间的误差，并使神经网络走子概率p与搜索概率π的相似度最大化。具体而言，参数θ通过梯度下降分别在均方误差和交叉熵损失之和上的损失函数l进行调整，

#### $(p,v)= f_θ (s)$,                            $l = (z - v)^2- π^T  log p + c||θ||^2$               (等式1)

其中c是控制L2权重正则化水平的参数（防止过拟合）。

## 2、AlphaGo Zero训练经验

我们使用强化学习来训练AlphaGo Zero。从随机下棋开始，没有人为干预，持续训练约3天。在训练过程中，生成了490万个自对弈棋局，每次MCTS使用1600次模拟，每次走子花费大约0.4s的思考时间。使用2048个局面的70万个mini-batches训练参数。神经网络包含20个残差块（详见[`方法`](#方法)）。

图3a显示了AlphaGo Zero在自我对弈强化学习期间的表现，作为训练时间和国际等级分的函数。学习在整个训练过程中顺利进行，并没有出现先前文献中提出的振荡或灾难性遗忘。令人惊讶的是，AlphaGo Zero仅仅36小时就打败了AlphaGo Lee；为了比较，AlphaGo Lee训练了几个月的时间。在72小时之后，我们根据首尔人机比赛中的比赛条件（见[`方法`](#方法)）和2小时时间控制，评估了AlphaGo Zero与击败Lee Sedol版本的AlphaGo Lee。AlphaGo Zero使用了一台带有4个TPU的机器，而AlphaGo Lee则分布在多台机器上，并使用了48个TPU。AlphaGo Zero以100比0击败了AlphaGo Lee（参见扩展数据图5和补充信息）。

![a3](assets\a3.png")
图3：AlphaGo Zero的训练评估。a.自我对弈强化学习的表现。该图显示了来自AlphaGo Zero强化学习的每次迭代i的每个MCTS棋手的表现$α_{θ_i }$。国际等级分是估算不同棋手间下棋计算出来的，每次走子使用0.4秒的思考时间（见[`方法`](#方法)）。为了比较，还显示了使用KGS数据集通过人类数据监督学习训练的棋手。b.人类走子的预测准确率。该图显示了神经网络$f_{θ_i }$在自我对弈i的每次迭代中，用于预测来自GoKifu数据集的人类走子的准确率。准确率度量的是神经网络为人类走子分配高概率的位置的百分比。也显示了通过监督学习训练的神经网络的准确率。c.在人类下棋结果的均方误差（MSE）。该图显示神经网络$f_{θ_i }$的MSE，在自我对弈i的每次迭代中，使用GoKifu数据集预测人类下棋的结果。MSE介于实际结果$z ∈ ${-1，+1}和神经网络价值v（胜率）之间，范围是[0,1]。还显示了通过监督学习训练的神经网络的MSE。

为了评估自我对弈强化学习的优点，与从人类数据中学习相比，我们训练了第二个神经网络（使用相同的体系结构）来预测KGS数据集中的专家走子；与之前的工作（分别参见扩展数据表1和2）相比，这实现了最好的预测精度。监督学习能够获得更好的初期表现，更好地预测人类专家下棋的结果（图3）。值得注意的是，尽管监督学习获得了更高的走子预测准确率，但是自学者的整体表现更好，在训练的前24小时内击败了学习人类的棋手。这表明AlphaGo Zero可能正在学习一种与人类有着本质区别的策略。

为了分别评估架构和算法的贡献，我们比较了AlphaGo Zero的神经网络架构与AlphaGo Lee使用的神经网络架构（见图4）的表现。使用AlphaGo Lee独立的策略和价值网络或者AlphaGo Zero中组合的策略和价值网络来创建四个神经网络；并使用AlphaGo Lee的卷积网络架构或AlphaGo Zero的残差网络架构。在自我对弈训练72小时后，使用由AlphaGo Zero产生的自我对弈的固定数据集来训练每个网络以最小化相同的损失函数（等式1）。使用残差网络更准确，达到更低的误差，AlphaGo的性能提高了600多个等级分。将策略和价值组合在一个网络中，略微降低了移动走子的准确率，但减少了价值错误，并提升了AlphaGo 600个等级分的下棋表现。这部分由于提高了计算效率，但更重要的是，将网络双重目标规范化为支持多个用例的共同表示。

![a4](assets\a4.png")
图4：AlphaGo Zero和AlphaGo Lee神经网络架构的比较。使用单独的（“sep”）或组合的策略和价值网络（“dual”）以及使用卷积（“conv”）或残差网络（“res”）的神经网络架构的比较。“dual-res”和“sep-conv”组合分别对应于AlphaGo Zero和AlphaGo Lee中使用的神经网络架构。每一个网络都是通过之前运行的AlphaGo Zero生成的固定数据集来训练的。a.每个训练过的网络都与AlphaGo Zero的搜索相结合，得到不同的棋手。国际等级分是根据这些不同的棋手之间的对弈计算出来的，每一步棋的思考时间为5秒。b.对每个网络架构在人类专家走子（来自GoKifu数据集）的预测准确率。c.每个网络架构的在人类专家下棋结果（来自GoKifu数据集）的均方误差。

## 3、AlphaGo Zero学到的知识

AlphaGo Zero在其自我对弈训练过程中发现了非凡的围棋知识。这包括人类围棋知识的基本要素，以及超出传统围棋知识范围的特殊策略。

图5显示了一个时间线，指示何时发现了专业的定式（托角序列）（图5a，扩展数据图1）；最终AlphaGo Zero更喜欢以前未发现的新定式（图5b，扩展数据图2）。图5c和补充信息显示了几个在不同训练阶段的快速自我对弈。AlphaGo Zero从完全随机的走子过渡到对围棋概念的完善理解，包括布局（开场），手筋（战术），死活，劫（重复棋局），缓气劫（终局），对杀，先手（主动 ），形状，势和地，都是从最基本规则中发现的。令人惊讶的是，Shocho（“征”吃序列可能跨越整个棋盘）是人类学习的围棋知识的第一个元素之一，后来在AlphaGo Zero的训练中得到了理解。

![a5](assets\a5.png")
图5：AlphaGo Zero学到的围棋知识。a.在AlphaGo Zero训练期间发现了五种人类定式（常见的托角序列）。相关的时间戳指示在自我对弈训练期间每个序列第一次发生的时间（考虑旋转和反转）。扩展数据图1提供了每个序列训练出现的频率。b.在自我训练的不同阶段偏爱的五个定式。在自我对弈训练的迭代中，每个显示的托角序列在所有托角序列中高频率出现。该迭代的时间戳在时间线上标识了出来。训练10个小时，弱托角是首选。训练47小时，点三三入侵使用最频繁。这个定式也是在人类的对弈中常见的;然而，AlphaGo Zero后来发现并选择了一个新的变着。扩展数据图2提供了所有五个频繁出现的序列和新变着。c.在不同的训练阶段进行的三个自我对弈的前80步棋，每次搜索使用1600次模拟（大约0.4s）。训练3小时，游戏贪婪地关注于吃子，很像人类初学者。在19小时关注于死活、势和地，即围棋的根本。在70小时，下棋表现非常完美平衡，涉及多场战斗和一次复杂的劫争，最终以白子多半子获胜。请参阅完整对弈的补充信息。

## 4、AlphaGo Zero的最终表现

随后，我们使用更大的神经网络和更长的持续时间将我们的强化学习应用于AlphaGo Zero的第二个实例。再次训练从完全随机下棋开始，持续约40天。在训练过程中，共产生了2900万次自我对弈棋局。从310万个mini-batches（2048个局面）更新参数。神经网络包含40个残差块。学习曲线如图6a所示。

我们使用与AlphaGo Fan，AlphaGo Lee和之前的几个围棋程序进行内部比赛来评估训练好的AlphaGo Zero。我们还与最强大的现有程序AlphaGo Master进行了比赛，该程序基于论文提供的算法和体系结构，但是利用人类数据和特征（请参阅[`方法`](#方法)）——在2017年1月的网络比赛中以60-0击败了最强的人类专业玩家。在我们的评估中，所有的程序都被允许每步棋5秒的思考时间；AlphaGo Zero和AlphaGo Master均部署在一台带有4个TPU的机器上；AlphaGo Fan和AlphaGo Lee分别分配了176个GPU和48个TPU。我们还包括一个基于AlphaGo Zero的raw neural network（不使用MCTS的网络）；这个网络只是以最大的概率选择如何走子。

![a6](assets\a6.png")
图6：AlphaGo Zero的表现。a. AlphaGo Zero的学习曲线，使用40 block的残差网络训练40天。该图显示了我们的强化学习算法的每次迭代i的每个棋手$α_{θ_i}$的表现。国际等级分是根据不同棋手之间的棋局评估来计算的，每个搜索使用0.4秒（参见[`方法`](#方法)）。b. AlphaGo Zero的最终表现。AlphaGo Zero使用40个残差块神经网络，训练了40天。该图显示了各个围棋程序的比赛结果：AlphaGo Zero，AlphaGo Master（在线比赛中击败顶级人类玩家60-0），AlphaGo Lee（击败李世石），AlphaGo Fan（击败樊麾）以及之前的围棋程序Crazy Stone，Pachi和GnuGo。每个程序每步棋都有5秒的思考时间。AlphaGo Zero和AlphaGo Master部署在单独一台机器在Google Cloud上；AlphaGo Fan和AlphaGo Lee分布式部署在机群上。还包括来自AlphaGo Zero的raw neural network，其直接选择网络输出$p_a$中的最大概率走子a，而不使用MCTS。程序以Elo国际等级分25为单位进行评估：200点差距对应于胜率75％。

图6b显示了每个程序在Elo国际等级分上的表现。未使用任何预测的raw neural network达到了3,055的Elo等级分。AlphaGo Zero获得了5,185的评分，而AlphaGo Master则为4,858，AlphaGo Lee为3,739，AlphaGo Fan为3,144。

最后，我们评估了AlphaGo Zero与 AlphaGo Master在2小时时间的100场比赛。AlphaGo Zero以 89：11赢得了比赛（参见扩展数据图6和补充信息）。

## 5、结论

我们的结果彻底地证明，即使在最具挑战性的领域中，纯强化学习方法也是完全可行的：在没有基本规则之外的领域知识的情况下，没有人类的例子或指导，就有可能训练成超人类的水平。此外，与对人类专家数据的训练相比，纯粹的强化学习方法只需要几个小时的训练时间，能够逐渐获得更好的表现。使用这种方法，AlphaGo Zero大幅度地击败了之前最强大的使用手工特征的人类数据进行训练的AlphaGo版本。

人类已经从几千年来数百万的棋局中积累了围棋知识，总结了通用的模式、术语和书籍。在几天的时间里，从“白板”开始，AlphaGo Zero能够重新发现围棋的许多知识，以及为最古老的游戏提供新见解的新颖策略。

# 方法

强化学习策略迭代是一种经典算法，通过轮流在策略评估（估算当前策略的价值函数 ）和策略改进（使用当前价值函数来产生更好的策略）之间产生一系列改进策略。策略评估的一个简单方法是从采样的结果中估计价值函数。策略改进的一个简单方法是关于价值函数贪婪地选择行动。在庞大的状态空间中，需要近似评估每个策略并表示其改进。

基于分类的强化学习使用简单的MonteCarlo搜索来改进策略。每个动作都会执行许多rollouts（快速走子）。具有最大均值的动作提供了一个积极的训练实例，而所有其他的动作提供了负面的训练实例；然后策略被训练为将动作分为正面或负面，并在随后的rollouts中使用。当τ →0时，这可以被视为AlphaGo Zero训练算法的策略组件的前身。

更近期的具体事例，基于分类的修改策略迭代（CBMPI），也通过将价值函数回归到剪枝rollout值来执行策略评估，类似于AlphaGo Zero的价值组件；这在俄罗斯方块的游戏中取得了最好的结果。然而，这个先前的工作仅限于使用手工特征的简单rollouts和线性函数逼近。

AlphaGo Zero自我对弈算法可以类似地被理解为近似策略迭代方案，其中MCTS被用于策略改进和策略评估。策略改进始于神经网络策略，根据该策略的建议执行MCTS，然后将更强大的搜索策略投影到神经网络的函数空间中。策略评估应用于（更强大的）搜索策略：自我对弈的结果也被投射回神经网络的函数空间。这些投影步骤通过训练神经网络参数分别匹配搜索概率和自我对弈结果来实现。

Guo等人还将MCTS的输出投影到神经网络中，或者通过对搜索值进行价值网络回归，或者通过对MCTS选择的动作进行分类。这种方法被用来训练神经网络来玩Atari游戏；然而，MCTS是固定的 - 没有策略迭代 - 并没有使用任何训练过的网络。

## 游戏中的自我对弈强化学习

我们的方法最直接适用于完全信息的零和游戏。我们遵循先前工作中描述的交替马尔可夫博弈的形式，指出基于价值或策略迭代的算法自然延伸到这个环境。

自我对弈强化学习以前已经应用于围棋游戏。NeuroGo使用神经网络来代表价值函数，使用基于围棋关于连，地和眼的知识的复杂架构。这个神经网络是由时序差分学习（temporal-difference learning）进行训练，在自我对弈中预测地。一个相关的方法，RLGO，代表替代特征的线性组合的价值函数，而不是由功能的线性组合，详尽地列举所有3 × 3模式的棋子；它是通过时序差分学习来预测自我对弈中的赢家。NeuroGo和RLGO达到了很弱的业余水平。

蒙特卡洛树搜索（MCTS）也可以被看作是自我对弈强化学习的一种形式。搜索树的节点包含在搜索期间遇到的局面的价值函数；这些价值被更新以预测模拟自我对弈的胜者。MCTS程序之前在围棋中已经取得了非常强大的业余水平，但是使用了大量的领域专业知识：基于手工特征的快速走子策略，通过运行模拟下棋（rollout）来评估局面，直到游戏结束；以及基于手工特征的树策略，该策略在搜索树中选择走子。

自我对弈强化学习方法在其他游戏中取得了高水平的表现：国际象棋，跳棋，西洋双陆棋，奥赛罗棋（又叫黑白棋、反棋(Reversi)、翻转棋），Scrabble（英语拼字游戏）和最近的扑克。在所有这些例子中，价值函数都是通过回归或时序差分学习从自我对弈生成的数据来训练的。经过训练的价值函数在alpha-beta搜索中被用作一个简单的蒙特卡洛搜索或反事实遗憾最小化（Counterfactual Regret Minimization）的评估函数。但是，这些方法使用手工输入特征或手工特征模板。此外，学习过程使用监督式学习来初始化权重，手动选择棋子价值的权重，手动限制动作空间，或使用预先存在的计算机程序作为训练对手或产生游戏记录。

许多最成功和广泛使用的强化学习方法首先在零和游戏的背景下引入：时序差分学习首先被引入在一个跳棋游戏程序，而MCTS被引入在围棋游戏。然而，非常相似的算法在视频游戏机器人，工业控制和在线推荐系统中被证明是非常有效的。

## AlphaGo版本

我们比较AlphaGo的三个不同版本：

1.	AlphaGo Fan是之前发行的2015年10月与Fan Hui对弈的一个程序。这个程序分布式部署在使用176个GPU的许多机器上。
2.	AlphaGo Lee是在2016年3月4月1击败李世石的程序。它与以前未发表的AlphaGo Fan在很多方面都很相似。但是，我们强调了几个关键差异，以促进公平比较。首先，价值网络是通过AlphaGo自我对弈的快速游戏的结果来训练的，而不是使用策略网络自我对弈；这个过程迭代了好几次 - 这是本文提出的白板算法的第一步。其次，策略和价值网络比原始论文中描述的要大 - 分别使用256个平面的12个卷积层，并且接受更多迭代的训练。该网络还分布在许多使用48个TPU而不是GPU的机器上，使其能够在搜索过程中更快地评估神经网络。
3.	AlphaGo Master是2017年1月份以60-0击败顶级人类选手的程序。它以前是未发表的，但是使用本文所述的相同的神经网络结构，强化学习算法和MCTS算法。然而，它使用与AlphaGo Lee相同的手工特征和rollouts，并且通过从人类数据监督学习来初始化训练。
4.	AlphaGo Zero是本文描述的程序。它从自我对弈强化学习中学习，从随机初始权重开始，不使用rollouts，没有人为的监督，只用历史棋局作为输入特征。它只使用一台在Google Cloud上的带有4个TPU的机器（AlphaGo Zero也可以分布式部署，但我们选择使用最简单的搜索算法）。


## 领域知识

我们的主要贡献是证明超人的表现可以在没有人类领域知识的情况下实现。

为了阐明这一贡献，我们列举了AlphaGo Zero在其训练过程或蒙特卡洛树搜索中明确或隐含地使用的领域知识；这些是AlphaGo Zero需要替换以学习不同的（交替马尔可夫）游戏的知识项目。

1.	AlphaGo Zero获得了完整的游戏规则知识。这些知识在MCTS中使用，模拟由一系列走子产生的局面，并对达到结束状态的任何模拟进行评分。当双方都放弃行棋（pass）时，或在19 · 19 · 2 = 722次走子后，游戏终止。此外，玩家在每个局面提供一组合法的走子。
2.	AlphaGo Zero在MCTS模拟和自我对弈训练中使用Tromp-Taylor评分。这是因为如果游戏在领土边界得到解决之前终止，那么人类的分数（中国，日本或者韩国的规则）就没有明确的定义。不过，所有比赛和游戏评估都是使用中国规则进行评分的。
3.	描述局面的输入特征被构造为19 × 19图像；即神经网络结构与棋盘的网格结构相匹配。
4.	围棋的规则在旋转和反转下是不变的； AlphaGo Zero中已经使用了这种知识，通过增加训练期间的数据集以包括每个局面的旋转和反转，并且在MCTS期间对局面的随机旋转或反转进行采样（参见[`搜索算法`](#搜索算法)）。除了贴目以外，围棋的规则也是颜色转换不变的。 这个知识是通过从当前玩家的角度来表示棋盘来使用的（见[`神经网络结构`](#神经网络结构)）。


除了上面列出的要点之外，AlphaGo Zero不使用任何形式的领域知识。它只使用其深层神经网络来评估叶节点并选择走子（参见下面的部分）。它不使用任何rollout策略或树策略，并且MCTS不会被任何其他启发式或特定于领域的规则增强。排除非法走子 - 即使是填充玩家自己的眼（以前所有程序中使用的标准启发式）。

该算法从神经网络的随机初始参数开始。神经网络体系结构（参见[`神经网络结构`](#神经网络结构)）基于当前的图像识别技术，并据此选择用于训练的超参数（参见[`自我对弈训练`](#自我对弈训练)）。通过高斯过程优化来选择MCTS搜索参数，以便使用在初步训练的神经网络来优化AlphaGo Zero的自我对弈表现。对于较大的运行（40块，40天），使用较小运行（20块，3天）训练的神经网络重新优化MCTS搜索参数。训练算法是在没有人为干预的情况下自主执行的。

## 自我对弈训练

AlphaGo Zero的自我对弈训练由三个主要组件组成，所有组件都是异步并行执行的。神经网络参数$θ_i$从最近的自我对弈数据持续优化；持续对AlphaGo Zero棋手 $α_{θ_i}$进行评估；迄今为止表现最好的棋手$α_{θ_*}$被用来产生新的自我对弈数据。

## 优化

每个神经网络$f_{θ_i}$在Google Cloud上使用TensorFlow进行优化，包括64个GPU worker和19个CPU参数服务器。batch-size为每个worker 32个，总的mini-batch为2,048个。从最近的50万次自我对弈的所有局面随机抽取每个mini-batch数据。神经网络参数使用动量和学习率退火随机梯度下降法，使用方程1中的损失进行优化。学习率按照扩展数据表3中的标准时间表进行退火。动量参数设置为0.9。交叉熵和均方误差损失的权重相同（这是合理的，因为奖励被归一化到$r ∈ ${−1, +1}），L2正则化参数设为$c = 10^{-4}$。优化过程每1,000个训练步骤产生一个新的检查点。这个检查点由评估者评估，并且可以用来生成下一个batch自我对弈数据，我们将在后面解释。

## 评估者

为了确保我们始终生成最好质量的数据，我们将每个新的神经网络检查点与当前最好的网络$f_{θ_*}$进行比较，然后将其用于数据生成。通过使用$f_{θ_i}$评估叶子节点的局面和先验概率（参见[`搜索算法`](#搜索算法)）的MCTS搜索$α_{θ_i}$的性能来评估神经网络$f_{θ_i}$。每个评估包括400场比赛，使用1,600个模拟的MCTS来选择每个走子，使用无限小的温度τ → 0（即我们确定性地选择最大访问次数的走子，以提供最强的可能性）。如果新玩家赢得55％以上的比赛（为了避免在单独噪音上选择），那么它将成为最好的玩家$α_{θ_*}$，随后用于生成自我对弈数据，并成为后续比较的基准。

## 自我对弈 

由评估者选择的当前最佳玩家$α_{θ_*}$被用于产生数据。在每次迭代中，$α_{θ_*}$进行25,000次自我对弈， MCTS搜索使用1,600次模拟来选择每次走子（每次搜索大约需要0.4s）。对于每场比赛的前30个走子，温度设置为τ = 1；这将根据MCTS中的访问次数按比例选择走子，并确保覆盖不同的局面（模拟不同的走法）。对于游戏的其余部分，使用无限小的温度τ→0。在根节点s0中的先验概率中添加Dirichlet噪声 $P(s,a) = (1 - ϵ)p_a  + ϵη_a$，其中η∼Dir（0.03）和ϵ = 0.25，实现额外的探索；这种噪音确保所有的走子都可以被尝试，但是搜索仍然可能下臭棋。为了节省计算，明显会输的游戏将投子认负。选择认输阈值$v_resign$以自动保持误报率（如果AlphaGo没有认输可能赢得的游戏比例）小于5％。为了测量误报（false positives），我们在10％的自我对弈中禁止认输，并下到游戏结束。

## 监督学习

为了比较，我们还通过监督学习来训练神经网络参数$θ_{SL}$。神经网络架构与AlphaGo Zero相同。从KGS数据集中随机抽取小批量（Mini-batches）的数据(s, π, z)，为人类专家走子a设定$π_a= 1$。通过动量和学习率退火的随机梯度下降，使用与方程1中相同的损失对参数进行优化，但是将均方误差系数设为0.01。学习率按照扩展数据表3中的标准时间表退火。动量参数设为0.9，L2正则化参数设为$c = 10^{-4}$。

通过使用组合的策略和价值网络体系结构，并且通过对价值组件使用较低的权重，可以避免价值的过拟合（在之前的工作中描述的问题）。72小时后，走子预测准确率超过了之前的网络，KGS测试集达到了60.4%；价值预测误差也大大好于以前。验证集由GoKifu的专业游戏数据组成。准确率和均方误差分别在扩展数据表1和扩展数据表2中报告。

## 搜索算法

AlphaGo Zero使用AlphaGo Fan和AlphaGo Lee中使用的非常简单的异步策略和价值MCTS算法（APV-MCTS）的变种。

搜索树中的每个节点都包含所有合法动作a ∈ A(s)的边(s, a)。 每个边缘存储一组统计信息，

{N(s, a), W(s, a), Q(s, a), P(s, a)},

其中N(s, a)是访问次数，W(s, a)是总行动价值，Q(s, a)是平均行动价值，P(s, a)选择边的先验概率。多个模拟分别在搜索线程上并行执行。该算法通过三个阶段（图2中的a-c）的迭代，然后选择走子（图2中的d）。

### Select（图2a）

选择阶段几乎与AlphaGo Fan相同；为了完整性在这里概括一下。每个模拟的第一个入树阶段开始于搜索树的根节点$s_0$，并且在时间步骤L模拟到达叶节点$s_L$时结束。在每个时间步t <L，根据搜索树中的统计值， $a_t  = argmax_a Q(s_t  ,a) + U(s_t  ,a)$  使用PUCT算法的变体来选择动作，

$U(s,a) = c_{puct} P(s,a)  \frac{\sqrt {\sum_b N(s,b)}}{1 + N(s,a)}$

$c_{puct}$是一个决定探索水平的常数；这种搜索控制策略最初倾向于具有高先验概率和低访问次数的行为，但是渐近地倾向于具有高行动价值的行为。

### Expand and evaluate (图2b)

叶节点$s_L$被添加到用于神经网络评估的队列中，$(d_i(p), v) = f_θ(d_i (s_L))$，其中$d_i$是从i∈ [1..8]的随机数均匀选择的棋盘反转或旋转。

在队列中的局面由神经网络使用最小批量mini-batch 大小为8进行评估；搜索线程被锁定，直到评估完成。叶子节点被展开，每个边($s_L$, a)被初始化为{$N(s_L, a) = 0，W(s_L, a) = 0，Q(s_L, a) = 0，P(s_L, a) = p_a$} 然后值v被回传（backed up）。

### Backup (图2c).

边的统计数据在每一步t≤L中反向更新。访问计数递增，$N(s_t , a_t) = N(s_t , a_t) +1$，并且动作价值更新为平均值， $W(s_t , a_t) = W(s_t , a_t) + v，Q(s_t , a_t) = \frac{W(s_t  ,a_t)}{N(s_t  ,a_t)}$。我们使用虚拟损失来确保每个线程评估不同的节点。

### Play (图2d)

在搜索结束时，AlphaGo Zero在根节点s0选择一个走子a，与其访问计数幂指数成正比，$π(a|s_0) = \frac{N(s_0,a) ^{1/τ}}{\sum_b N(s_0,b)^{1/τ}}$ ，其中τ是控制探索水平的温度参数。在随后的时间步重新使用搜索树：与所走子的动作对应的子节点成为新的根节点；保留这个节点下面的子树所有的统计信息，而树的其余部分被丢弃。如果根节点的价值和最好的子节点价值低于阈值$v_{resign}$，则AlphaGo Zero会认输。

与AlphaGo Fan和AlphaGo Lee的MCTS相比，主要区别在于AlphaGo Zero不使用任何rollouts（快速走子）；它使用单一的神经网络，而不是独立的策略和价值网络；叶节点总是被扩展，而不是使用动态扩展；每个搜索线程需要等待神经网络的评估，而不是异步地进行评估和backup；没有树策略（是选择节点(落子)的策略）。在AlphaGo Zero的大型（40块，40天）实例中也使用了转移表。

## 神经网络结构

神经网络的输入是包含17个二值（只有两个值0和1）特征平面的19×19×17维度的图像栈。8个特征平面$X_t$由二进制值组成，表示当前玩家存在的棋子（如果交点i在时间步t包含玩家颜色的棋子，那么$X_t^i = 1$；如果交叉点是空的，包含对手棋子，或者t <0，$X_t^i  = 0$）。另外8个特征平面$Y_t$表示对手的棋子的相应特征。最后的特征面C表示棋子颜色（当前的棋盘状态），是常量，如果是黑色棋子，则为1，如果是白色棋子则为0。这些平面连接在一起，给出输入特征$s_t = [X_t , Y_t , X_{t-1}, Y_{t-1}, ..., X_{t-7}, Y_{t-7}, C]$。历史特征$X_t$，$Y_t$是必要的，因为围棋不能完全从当前的棋子中观察到这些历史，重复是被禁止的。类似地，颜色特征C也是必要的，因为贴目是不可观察的。

输入特征$s_t$由一个残差塔处理，该塔由单个卷积块组成，后跟19个或39个残差块。

卷积块应用以下模块：
1.	卷积层：256 filters， kernel size 3 × 3 ，stride 1
2.	Batch normalisation 
3.	整流非线性ReLU


每个残差块将下列模块按顺序应用于其输入：
1.	卷积层：256 filters， kernel size 3 × 3 ，stride 1
2.	Batch normalization
3.	整流非线性ReLU
4.	卷积层：256 filters ，kernel size 3 × 3 ，stride 1
5.	Batch normalization
6.	将输入直接连接到块
7.	整流非线性ReLU


残差塔的输出分别传递到两个单独的“头”，分别计算策略和价值。 策略“头”应用以下模块：
1.	卷积层：2 filters ， kernel size 1 × 1 ，stride 1
2.	Batch normalization
3.	整流非线性ReLU
4.	一个全连接的线性层，输出一个大小为$19^2 + 1 = 362$的向量，对应于所有交叉点的logit概率（每个点的走子概率）。


价值“头”应用了以下模块：
1.	卷积层： 1 filter ， kernel size 1 × 1 ，stride 1
2.	Batch normalization
3.	整流非线性ReLU
4.	一个全连接的线性层到一个大小为256的隐层
5.	整流非线性ReLU
6.	一个全连接的线性层到一个标量
7.	tanh激活函数：输出范围在[-1,1]的标量


在20或40块网络中，整个网络深度分别为残差塔的39或79个参数化层，另外还有2层用于策略头，3层用于价值头。

我们注意到残差网络的不同变体同时应用于计算机围棋，并实现了业余级的水平；然而，这仅限于由监督学习单独培训的单一策略网络。

## 神经网络体系结构比较

图4显示了网络架构之间的比较结果。具体来说，我们比较了四种不同的神经网络：
1.	dual-res:如上所述，该网络包含一个20 block的残差塔，其后是一个策略头和一个价值头。 这是AlphaGo Zero中使用的架构。
2.	sep-res:该网络包含两个20 block残差塔。 第一个塔后面是策略头，第二个塔后面是一个价值头。
3.	dual-conv:该网络包含一个12个卷积块的非残差塔，其后是一个策略头和一个价值头。
4.	sep-conv:该网络包含两个12个卷积块的非残差塔。 第一个塔后面是策略头，第二个塔后面是一个价值头。 这是AlphaGo Lee使用的架构。


每个网络都是在一个固定的数据集上进行训练的，这个数据集包含了以前运行的AlphaGo Zero生成的最后200万个自我对弈数据，使用带有退火速率、动量和监督学习实验描述的正则化超参数的随机梯度下降；交叉熵和均方误差分量使用相同的权重，因为有更多的数据可用。

## 评估

我们通过测量每个玩家的国际等级分来评估AlphaGo Zero的相对强度（图3a和图6）。我们通过logistic函数p(a defeats b) = $\frac{1}{1+exp (c_{elo} (e(b)-e(a))}$估计玩家a击败玩家b的概率，并且通过由BayesElo程序使用标准常数$c_{elo} = 1/400$计算贝叶斯logistic回归估计评级e(·)。

Elo等级分是根据AlphaGo Zero，AlphaGo Master，AlphaGo Lee和AlphaGo Fan之间每5秒一步棋的比赛结果计算出来的。来自AlphaGo Zero的裸神经网络（不包含MCTS）也包含在比赛中。

AlphaGo Fan，Crazy Stone，Pachi和GnuGo的Elo等级分是根据之前工作的比赛价值来确定的，并与该工作中提出的棋手相对应。AlphaGo Fan对樊麾和AlphaGo Lee对李世石的比赛结果也包括在内，以此作为人类参考的依据，否则AlphaGo的Elo等级分由于自我对弈偏差而不切实际的高。

图3a，4a和6a中的Elo等级分是根据自我对弈训练期间玩家$α_{θ_i}$的每次迭代之间的评估游戏的结果计算的。进一步的评估也是针对Elo等级分的基线玩家进行的。

我们测量了AlphaGo Zero与AlphaGo Lee比赛的表现，以及有40个block的AlphaGo Zero与AlphaGo Master的比赛，使用了与2016年首尔对李世石使用的相同的版本和比赛条件。每个玩家接受2个小时的思考时间加上每个走子60秒的3个读秒时间。所有的比赛都使用7.5分的贴目的中国规则评分。

## 数据可用性

用于验证和测试的数据集是[`GoKifu数据集`](http://gokifu.com/)和[`KGS数据集`](https://u-go.net/gamerecords/获得)。
![a7](assets\a7.png")
扩展数据表1：走子预测准确率。分别通过强化学习（即AlphaGo Zero）或监督学习训练的神经网络的走子预测的精度百分比。对于监督学习，网络在KGS数据（业余比赛）上接受了3天的训练。对于强化学习，20个block网络训练了3天，40个block网络训练了40天。还根据基于专业比赛的GoKifu数据集作为验证集对网络进行了评估。

![a8](assets\a8.png")
扩展数据表2：比赛结果预测误差。分别对通过强化学习（即AlphaGo Zero）或监督学习训练的神经网络的比赛结果预测的均方误差。对于监督学习，网络在KGS数据（业余比赛）上接受了3天的训练。对于强化学习，20个block网络训练了3天，40个block网络训练了40天。还根据基于专业比赛的GoKifu数据集作为验证集对网络进行了评估。


![a9](assets\a9.png")
扩展数据表3：学习率计划。在强化学习和监督学习实验中使用的学习率，以千步（小批量mini-batch更新）测量。


![a10](assets\a10.png")
扩展数据图1：对于图5a中的每个定式（AlphaGo Zero发现的专业比赛中常见的托角序列），在训练期间随时间发生的频率。相应的定式作为插图在本图中显示。

![a11](assets\a11.png")
扩展数据图2：训练期间随时间发生的频率，对于图5b中的每个定式（AlphaGo Zero偏爱至少一次迭代的托角序列）以及一个额外的变着。相应的定式作为插图在本图中显示。